## 3장: 어텐션 메커니즘 구현하기

In [1]:
from importlib.metadata import version
print("파이토치 버전:", version("torch"))

파이토치 버전: 2.6.0


In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
print("input shape: ", inputs.shape)

input shape:  torch.Size([6, 3])


In [4]:
query = inputs[1]   # 두 번째 입력 토큰이 쿼리입니다.
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
res = 0
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]
print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **단계 2**: 정규화 되지 않은 어텐션 점수(ω)를 합이 1이 되도록 정규화합니다.

In [6]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("어텐션 가중치: ", attn_weights_2_tmp)
print("합: ", attn_weights_2_tmp.sum())

어텐션 가중치:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합:  tensor(1.0000)


In [7]:
def softmax_navie(x):
    return torch.exp(x)/ torch.exp(x).sum()

attn_weights_2_navie = softmax_navie(attn_scores_2)
print("어텐션 가중치: ", attn_weights_2_navie)
print("합: ", attn_weights_2_navie.sum())

어텐션 가중치:  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합:  tensor(1.)


In [8]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("어텐션 가중치:", attn_weights_2)
print("합:", attn_weights_2.sum())

어텐션 가중치: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
합: tensor(1.)


In [9]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])
